### Tutorial on how to use the train/test method listed in the s2s `traintest` module

For cross-validation, we choose the train/test splitting method in the s2s `time` module and get the split groups.

We start by importing the required libraries and generating an example `AdventCalendar`.

In [1]:
import s2spy.time

In [2]:
calendar = s2spy.time.AdventCalendar(anchor_date=(10, 15), freq='180d')
intervals = calendar.map_years(2019, 2021).flat
intervals

anchor_year  i_interval
2021         0             (2021-04-18, 2021-10-15]
             1             (2020-10-20, 2021-04-18]
2020         0             (2020-04-18, 2020-10-15]
             1             (2019-10-21, 2020-04-18]
2019         0             (2019-04-18, 2019-10-15]
             1             (2018-10-20, 2019-04-18]
dtype: interval

Example of `kfold` method.

In [3]:
calendar.set_traintest_method("kfold", n_splits = 2, shuffle = False)

Get train/test groups.

In [4]:
traintest_group = calendar.traintest
traintest_group

,,,0,1
anchor_year,fold_0,fold_1,,
2021,test,train,"(2021-04-18, 2021-10-15]","(2020-10-20, 2021-04-18]"
2020,test,train,"(2020-04-18, 2020-10-15]","(2019-10-21, 2020-04-18]"
2019,train,test,"(2019-04-18, 2019-10-15]","(2018-10-20, 2019-04-18]"


### xarray example

In [5]:
from s2spy import traintest
import numpy as np
import pandas as pd

In [6]:
# Generate example data
time_index = pd.date_range("20171001", "20211001", freq="60d")
test_data = np.random.random(len(time_index))
expected = np.array([test_data[4:7].mean(), test_data[1:4].mean()])
series = pd.Series(test_data, index=time_index, name='data1')
df = pd.DataFrame(series)
ds = df.to_xarray().rename({'index': 'time'})

In [7]:
cal = s2spy.time.AdventCalendar(freq="180d")
ds_resampled = cal.resample(ds)
s2spy.traintest.kfold(ds_resampled)

<xarray.Dataset>
Dimensions:      (anchor_year: 3, i_interval: 2)
Coordinates:
    index        (anchor_year, i_interval) int64 0 1 2 3 4 5
    interval     (anchor_year, i_interval) object (2018-06-03, 2018-11-30] .....
  * anchor_year  (anchor_year) int64 2018 2019 2020
  * i_interval   (i_interval) int64 0 1
    target       (i_interval) bool True False
    fold_0       (anchor_year) <U6 'test' 'test' 'train'
    fold_1       (anchor_year) <U6 'train' 'train' 'test'
Data variables:
    data1        (anchor_year, i_interval) float64 0.4285 0.3228 ... 0.7531